In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
data=sns.load_dataset("diamonds")

In [3]:
data.shape

(53940, 10)

In [4]:
data.shape

(53940, 10)

In [5]:

data=data.rename(columns={"x":"length","y":"width","z":"height"})

In [6]:
data["price"]=data["price"]*83.04

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline,Pipeline

In [8]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [9]:
d=data[["price"]]


In [10]:
data.drop(labels=["price"],axis=1,inplace=True)

In [11]:
data["price"]=d
data.head()

,carat,cut,color,clarity,depth,table,length,width,height,price
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,27071.04
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,27071.04
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,27154.08
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,27735.36
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,27818.40


In [12]:
x=data.iloc[:,:-1]
y=data.iloc[:,-1:]

In [13]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [14]:

from sklearn import set_config
set_config(display="diagram")


In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# ON RandomForestRegressor

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
trf1 = ColumnTransformer([
    ("trf1", OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
                                        ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                                        ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1']]), [1, 2, 3])
], remainder='passthrough')
trf2 = ColumnTransformer([("trf2", StandardScaler(), slice(0, 10))])
trf3=RandomForestRegressor()

In [18]:
pipe1 = Pipeline(steps=[
    ('columntransformer1', trf1),
    ('columntransformer2', trf2),
    ('randomforestreeregressor', RandomForestRegressor())  # Ensure this is the correct step name
])

In [19]:

n_estimators=[20,40,60,80,100,120]
max_features=[0.2,0.6,0.8,1.0]
max_depth=[2,3,5,None]
max_sample=[0.4,0.5,0.75,0.95,1.0]
bootstrap=[True,False]
min_sample_split=[2,3,5]
min_sample_leaf=[1,2]

In [20]:
param_grid={'randomforestreeregressor__n_estimators':n_estimators,'randomforestreeregressor__max_features':max_features,
            'randomforestreeregressor__max_depth':max_depth,'randomforestreeregressor__max_samples':max_sample,
            'randomforestreeregressor__bootstrap':bootstrap,'randomforestreeregressor__min_samples_split':min_sample_split,
            'randomforestreeregressor__min_samples_leaf':min_sample_leaf
           }

In [21]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
rf_grd=RandomizedSearchCV(estimator=pipe1,)

In [22]:
rf_grid=RandomizedSearchCV(pipe1,param_distributions=param_grid,cv=5,n_jobs=-1)

In [23]:
rf_grid.fit(X_train,Y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('columntransformer1',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('trf1',
                                                                               OrdinalEncoder(categories=[['Fair',
                                                                                                           'Good',
                                                                                                           'Very '
                                                                                                           'Good',
                                                                                                           'Premium',
                                                                                                           'Ideal'],
                                                                                                          ['D',
                                                                                                           'E',
                                                                                                           'F',
                                                                                                           'G',
                                                                                                           'H',
                                                                                                           'I',
                                                                                                           'J'],
                                                                                                          ['IF',
                                                                                                           'VVS1',
                                                                                                           'VVS2',
                                                                                                           'VS1',
                                                                                                           'VS2',
                                                                                                           'SI1',
                                                                                                           'SI2',
                                                                                                           'I1']]),
                                                                               [1,
                                                                                2,
                                                                                3])])),
                                             ('columntransformer2',
                                              ColumnTransformer(transf...
                                                                                False],
                                        'randomforestreeregressor__max_depth': [2,
                                                                                3,
                                                                                5,
                                                                                None],
                                        'randomforestreeregressor__max_features': [0.2,
                                                                                   0.6,
                                                                                   0.8,
                                                                                   1.0],
                                        'randomforestreeregressor__max_samples': [0.4,
                                                                            

In [24]:
rf_grid.best_params_

{'randomforestreeregressor__n_estimators': 80,
 'randomforestreeregressor__min_samples_split': 2,
 'randomforestreeregressor__min_samples_leaf': 1,
 'randomforestreeregressor__max_samples': 0.5,
 'randomforestreeregressor__max_features': 0.6,
 'randomforestreeregressor__max_depth': None,
 'randomforestreeregressor__bootstrap': True}

In [25]:
y_prr=rf_grid.predict(X_test)

In [26]:
r2_score(Y_test,y_prr)

0.9817896650799789

In [30]:
trf_s=RandomForestRegressor(n_estimators=80,min_samples_split=2,min_samples_leaf=1,max_samples=0.5,max_features=0.6,max_depth=None,bootstrap=True)

In [31]:
pipe2=make_pipeline(trf1,trf2,trf_s)

In [32]:
pipe2.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf1',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['IF',
                                                                              'VVS1',
                                                                              'VVS2',
                                                                              'VS1',
                                                                              'VS2',
                                                                              'SI1',
                                                                              'SI2',
                                                                              'I1']]),
                                                  [1, 2, 3])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('trf2', StandardScaler(),
                                                  slice(0, 10, None))])),
                ('randomforestregressor',
                 RandomForestRegressor(max_features=0.6, max_samples=0.5,
                                       n_estimators=80))])

In [33]:
y_t=pipe2.predict(X_test)

In [34]:
r2_score(Y_test,y_t)

0.9817510336727291

In [35]:
test=np.array([2.0,	"Ideal"	,"G","SI2",	62.2	,57.0	,8.13,	8.0,	5.02],dtype=object).reshape(1,9)

In [36]:
pipe2.predict(test)

array([1235286.432])

In [47]:
pickle.dump(pipe2,open('pipe2.pkl','wb'))

In [ ]:
# ON DECESISIONTREE----

# New section

In [37]:
trf1 = ColumnTransformer([
    ("trf1", OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
                                        ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                                        ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1']]), [1, 2, 3])
], remainder='passthrough')
trf2 = ColumnTransformer([("trf2", StandardScaler(), slice(0, 10))])

trf3=DecisionTreeRegressor(criterion="squared_error",max_depth=8,max_features='auto',splitter='best')

In [38]:
pipe = make_pipeline(trf1,trf2,trf3)

In [39]:
pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf1',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['IF',
                                                                              'VVS1',
                                                                              'VVS2',
                                                                              'VS1',
                                                                              'VS2',
                                                                              'SI1',
                                                                              'SI2',
                                                                              'I1']]),
                                                  [1, 2, 3])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('trf2', StandardScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=8, max_features='auto'))])

In [40]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf1',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['IF',
                                                                              'VVS1',
                                                                              'VVS2',
                                                                              'VS1',
                                                                              'VS2',
                                                                              'SI1',
                                                                              'SI2',
                                                                              'I1']]),
                                                  [1, 2, 3])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('trf2', StandardScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=8, max_features='auto'))])

In [41]:
y_pred=pipe.predict(X_test)

In [42]:
r2_score(y_pred,Y_test)

0.9688217531834243

In [43]:
pipe.set_output()

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf1',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['IF',
                                                                              'VVS1',
                                                                              'VVS2',
                                                                              'VS1',
                                                                              'VS2',
                                                                              'SI1',
                                                                              'SI2',
                                                                              'I1']]),
                                                  [1, 2, 3])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('trf2', StandardScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=8, max_features='auto'))])

In [44]:
test=np.array([2.0,	"Ideal"	,"G","SI2",	62.2	,57.0	,8.13,	8.0,	5.02],dtype=object).reshape(1,9)

In [45]:
pipe.predict(test)

array([1331731.19323944])

In [46]:
import pickle

In [ ]:
pickle.dump(pipe,open('pipe1.pkl','wb'))

In [ ]:
data.sample()

,carat,cut,color,clarity,depth,table,length,width,height,price
23892,2.0,Ideal,G,SI2,62.2,57.0,8.13,8.0,5.02,995483.52
